# Part 2: Evaluating our LLM application

So far, we've chosen typical/arbitrary values for the various parts of our RAG application. But if we were to change something, such as our chunking logic, embedding model, LLM, etc. how can we know that we have a better configuration than before. A generative task like this is very difficult to quantitatively assess and so we need to develop creative ways to do so. 

Because we have many moving parts in our application, we need to perform unit/component and end-to-end evaluation. Component-wise evaluation can involve evaluating our retrieval in isolation (is the best source in our set of retrieved chunks) and evaluating our LLMs response (given the best source, is the LLM able to produce a quality answer). As for end-to-end evaluation, we can assess the quality of the entire system (given all data, what is the quality of the response).

<img width="1000" src="https://images.ctfassets.net/xjan103pcp94/17UQdsEImsXOOdDlT06bvi/4a9b9e46e157541a1178b6938624176a/llm_evaluations.png">

## Setup

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = ...

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Golden Context Dataset

In an ideal world, we would have a golden validation dataset: given a set of queries, we would have the correct sources that answer those queries, and optionally the correct answer that should be returned by the LLM.

For this example, we have manually collected 177 representative user queries and identified the correct source in the documentation that answer those user queries.

In [ ]:
from pathlib import Path
import json

golden_dataset_path = Path("../datasets/eval-dataset-v1.jsonl")

with open(golden_dataset_path, "r") as f:
    data = [json.loads(item) for item in list(f)]
    
len(data)

Our dataset contains 'question' and 'source' pairs. If we have a golden context dataset, it is the best option for evaluation.

In [ ]:
data[:5]

## Cold Start

We may not always have a prepared dataset of questions and the best source to answer that question readily available. To address this cold start problem, we could use an LLM to look at our documents and generate questions that the specific chunk would answer. This provides us with quality questions and the exact source the answer is in. However, this dataset generation method could be a bit noisy. The generate questions may not always be resembling of what your users may ask and the specific chunk we say is the best source may also have that exact information in other chunks. Nonetheless, this is a great way to start our development process while we collect + manually label a high quality dataset.

<img width="800" src="https://images.ctfassets.net/xjan103pcp94/3QR9zkjtpgeqK8XKPteTav/76aa9e7743330e7fcf73b07332a7ddf2/image10.png">

We need to define a few parameters first.  
- Notably, the chunk size determines the size of the text chunk shown to the LLM when generating hypothetical question & answer pairs. This must be set below the context window limitation of the chosen LLM.
- We choose a subsample ratio since we just want to construct a small representative subset for the purpose of evaluation and iteration. (We choose an even smaller subset for the purpose of the demonstration here).
- We use `gpt-3.5-turbo` since it's fast and cheap. 

In [ ]:
from pathlib import Path

RAY_DOCS_DIRECTORY = Path("docs.ray.io/en/master/")

First, we load in the documents and chunk them to the appropriate sizes, creating LlamaIndex nodes. We already did the data processing in part 1, and have packaged the logic as a utility.

In [ ]:
from data import create_nodes

# needs to be smaller than context window
CHUNK_SIZE = 1024

nodes = create_nodes(RAY_DOCS_DIRECTORY, chunk_size=CHUNK_SIZE, chunk_overlap=20).take_all()

In [ ]:
nodes = [node_dict["node"] for node_dict in nodes]
id_to_node = {node.node_id: node for node in nodes}

Now, we subsample the nodes to obtain a representative subset (here we use a very small subset for a fast demonstration)

In [ ]:
from utils import subsample

SUBSAMPLE_RATIO = 0.01

subsampled_nodes = subsample(nodes, SUBSAMPLE_RATIO)
print('Subsampled {} nodes into {} nodes'.format(len(nodes), len(subsampled_nodes)))

Now, we use LlamaIndex's built in utility `generate_qa_embedding_pairs` to create synthetic query/context pairs.

(We can also use this utility for fine-tuning embeddings, hence the naming. More on this in part 3!)

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo')
synthetic_dataset = generate_qa_embedding_pairs(subsampled_nodes, llm=llm, num_questions_per_chunk=2)

Now we will transform the shape of the data a bit to match the format of our labeled data.

In [ ]:
synthetic_data = []
for query_id, context_ids in synthetic_dataset.relevant_docs.items():
    query = synthetic_dataset.queries[query_id]
    golden_context = id_to_node[context_ids[0]].metadata['source']
    entry = {
        'question': query,
        'source': golden_context,
    }
    synthetic_data.append(entry)

In [ ]:
synthetic_data[:5]

In [ ]:
from utils import write_jsonl

write_jsonl("../datasets/synthetic-eval-dataset.jsonl", synthetic_data)

Since we already have a dataset with representative user queries and ground truth labels, we will use that for evaluation instead of a synthetically generated dataset.

## Evaluating Retrieval

The first component to evaluate in our RAG application is retrieval. Given a query, is our retriever pulling in the correct context to answer that query? Regardless of how good our LLM is, if it does not have the right context to answer the question, it cannot provide the right answer.

We can use our golden context dataset to evaluate retrieval. The simplest approach is that for each query in our dataset, we can test to see if the correct source is included in any of the chunks that are retrieved by our retriever. This measures "hit rate".

However, simply checking for existence can be misleading if we increase the number of chunks that we retrieve. Therefore, we also want to check the score that our retriever gives for the correct source. A higher score means our retriever is accurately determining the correct context. 

To summarize, for each query in our evaluation dataset, we will measure the following:
1. Is the correct source included in any of the retrived chunks?
2. What is the score our retriever gives to the correct source?

<img width="800" src="../images/retrieval-eval.png">

First, let's a get a retriever over the vector database. We have packaged this as a utility. It is the same as we did in notebook 1.

In [ ]:
from utils import get_retriever

In [ ]:
retriever = get_retriever(similarity_top_k=5, embedding_model_name='text-embedding-ada-002')

Now let's evaluate our retriever. 

In [ ]:
from tqdm import tqdm
results = []

for entry in tqdm(data):
    query = entry["question"]
    expected_source = entry['source']
    
    retrieved_nodes = retriever.retrieve(query)
    retrieved_sources = [node.metadata['source'] for node in retrieved_nodes]
    
    # If our label does not include a section, then any sections on the page should be considered a hit.
    if "#" not in expected_source:
        retrieved_sources = [source.split("#")[0] for source in retrieved_sources]
    
    if expected_source in retrieved_sources:
        is_hit = True
        score = retrieved_nodes[retrieved_sources.index(expected_source)].score
    else:
        is_hit = False
        score = 0.0
    
    result = {
        "is_hit": is_hit,
        "score": score,
        "retrieved": retrieved_sources,
        "expected": expected_source,
        "query": query,
    }
    results.append(result)

In [ ]:
results[:2]

Let's see how well our retriever does. It's not great right now, but we now have a solid metric to evaluate our retriever for future optimizations.

In [ ]:
total_hits = sum(result["is_hit"] for result in results)
hit_percentage = total_hits / len(results)
hit_percentage

In [ ]:
average_score = sum(result["score"] for result in results) / len(results)
average_score

## End-to-end evaluation

While we can evaluate our retriever in isolation, ultimately we want to evaluate our RAG application end-to-end, which includes the final response generated from our LLM.

To effectively evaluate our generated responses, we need "ground truth" responses. These ground truth responses can be generated by feeding the correct context to a "golden" LLM. Then, we can use an LLM to evaluate our generated responses compared to the ground truth responses.

<img width="700" src="https://images.ctfassets.net/xjan103pcp94/2lhpSUNrMmi7WAHpd3wslR/15facf649e30571e8d806d354f475f0b/image6.png">

### Choosing a Golden LLM

To generate ground truth responses, and then to evaluate the generated responses vs. the ground truth, we need a "golden" LLM. But which LLM should we use? We now run into a problem: we need to determine the quality of different LLMs to choose as a "golden" LLM, but doing so requires a "golden" LLM. 

Leaderboards on general benchmarks provide a rough indication on which LLMs perform better, we will go with OpenAI's GPT-4 here since it's been shown to be [well aligned with human preferences](https://arxiv.org/pdf/2306.05685.pdf)

In [ ]:
from bs4 import BeautifulSoup

def fetch_text_from_source(source: str):
    url, anchor = source.split("#") if "#" in source else (source, None)
    file_path = Path("./", url.split("https://")[-1])
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    if anchor:
        target_element = soup.find(id=anchor)
        if target_element:
            text = target_element.get_text()
        else:
            return fetch_text_from_source(source=url)
    else:
        text = soup.get_text()
    return text

In [ ]:
example_source = data[0]["source"]
print(example_source)

text = fetch_text_from_source(example_source)
text

In [ ]:
from tqdm import tqdm
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.schema import TextNode, NodeWithScore

def generate_responses(entries, llm):
    context_window = llm.metadata.context_window - 500
    service_context = ServiceContext.from_defaults(llm=llm, context_window=context_window)
    rs = get_response_synthesizer(service_context=service_context)

    responses = []
    for entry in tqdm(entries):
        query = entry["question"]
        source = entry["source"]

        context = fetch_text_from_source(source)
        nodes = [NodeWithScore(node=TextNode(text=context))]

        response = rs.synthesize(query, nodes=nodes)
        responses.append(response.response)
    return responses

We can now generate our reference responses. Let's generate 10 reference responses and save them to a file.

In [ ]:
llm = OpenAI(model='gpt-4', temperature=0.0)
ten_samples = data[:10]
golden_responses = generate_responses(ten_samples, llm)

In [ ]:
reference_dataset = [{"question": entry["question"], "source": entry["source"], "response": response} for entry, response in zip(ten_samples, golden_responses)]

In [ ]:
with open("../datasets/golden-responses.json", "w") as file:
    json.dump(reference_dataset, file, indent=4)

## Evaluating our Query Engine

Once we have reference responses, we can get our generated responses from our query engine. Then pass both responses to our golden LLM to evaluate the responses from our application.

In [ ]:
with open("../datasets/golden-responses.json", "r") as file:
    golden_responses = json.load(file)

In [ ]:
golden_responses[0]

In [ ]:
from utils import get_query_engine

In [ ]:
query_engine = get_query_engine(similarity_top_k=5, llm_model_name='gpt-3.5-turbo', embedding_model_name='text-embedding-ada-002')

# Store both the original response object and the response string.
rag_responses = []
rag_response_str = []

for entry in tqdm(golden_responses):
    query = entry["question"]
    response = query_engine.query(query)
    rag_responses.append(response)
    rag_response_str.append(response.response)

In [ ]:
rag_response_str[0]

In [ ]:
from llama_index.evaluation import CorrectnessEvaluator

In [ ]:
eval_llm = OpenAI(model='gpt-4', temperature=0.0)
service_context = ServiceContext.from_defaults(llm=eval_llm)
evaluator = CorrectnessEvaluator(service_context=service_context)

In [ ]:
eval_results = []
for rag_response, golden_response in tqdm(list(zip(rag_response_str, golden_responses))):
    query = golden_response["question"]
    golden_answer = golden_response["response"]
    generated_answer = rag_response
    
    eval_result = evaluator.evaluate(query=query, reference=golden_answer, response=generated_answer)
    eval_results.append(eval_result)

In [ ]:
[r.score for r in eval_results]

Let's save the query, both responses, and the score to a JSON file

In [ ]:
scores = [
    {"question": golden_response["question"],
     "golden_response": golden_response["response"],
     "generated_response": eval_result.response,
     "score": eval_result.score,
     "reasoning": eval_result.feedback,
    }
    for eval_result, golden_response in zip(eval_results, golden_responses)
]

In [ ]:
with open("eval-scores.json", "w") as file:
    json.dump(scores, file, indent=4)

We can also calculate the average scores

In [ ]:
average_scores = sum(score["score"] for score in scores) / len(scores)
average_scores

## Evaluation without Golden Responses

Generating reference responses and then using them for evaluation can give us a more accurate assesment on how our query engine is performing. However, this approach can be expensive. We have to make an initial pass through GPT4 to generate the reference response, and then we have to make another pass through GPT4 to evaluate our application's responses against the reference response.

We can explore other evaluation metrics to get a better sense on how our query engine is performing, without needing to make multiple passes to GPT4.

### Evaluating for faithfulness/relevancy

One metric we can test is relevancy, which does not require generating reference responses. With this approach, we check to see if the generated response is relevant to at least one of the retrieved sources and to the query. This ensures that our LLM is not making up a response, but rather that it is relevant to the question that is being asked, and also that is relevant to at least one of the retrieved context.

This does NOT check whether the response is a correct response.

This capability is built into LlamaIndex, via the various `Evaluator` modules. We use gpt-4 as the evaluator.

In [ ]:
from llama_index.evaluation import FaithfulnessEvaluator, RelevancyEvaluator
from llama_index.llms import OpenAI
from llama_index import ServiceContext

def evaluate(queries: list, responses: list, metric: str):
    llm = OpenAI(model="gpt-4", temperature=0.0)
    service_context = ServiceContext.from_defaults(llm=llm)
    
    
    if metric == 'faithfulness':
        evaluator = FaithfulnessEvaluator(service_context=service_context)
    elif metric == 'relevancy':
        evaluator = RelevancyEvaluator(service_context=service_context)
    else:
        raise ValueError("Unknown metric: ", metrc)

    evals = []
    for query, response in tqdm(list(zip(queries, responses))):
        eval_result = evaluator.evaluate_response(query=query, response=response)
        evals.append(eval_result)
    
    return evals

def get_pass_rate(evals):
    return len([val.passing for val in evals]) / len(evals)

In [ ]:
faithfulness_results = evaluate(queries=[sample["question"] for sample in ten_samples], responses=rag_responses, metric='faithfulness')

In [ ]:
faithfulness_score = get_pass_rate(faithfulness_results)
faithfulness_score

In [ ]:
relevancy_results = evaluate(queries=[sample["question"] for sample in ten_samples], responses=rag_responses, metric='relevancy')

In [ ]:
relevancy_score = get_pass_rate(relevancy_results)
relevancy_score